# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
import os
import shutil
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
ws.write_config(path='.azureml')
experiment_name = 'Titanic_project'

experiment = Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-136380
Azure region: southcentralus
Subscription id: 2c48c51c-bd47-40d4-abbe-fb8eabd19c8c
Resource group: aml-quickstarts-136380


In [4]:
cpu_cluster_name = 'ML-cluster-uda'

try :
    cpu_cluster = ComputeTarget(workspace = ws,
                               name = cpu_cluster_name)
    print("Using previously created cpu_cluster")
except ComputeTargetException :
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2',
                                                          max_nodes = 4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output = True)

Using previously created cpu_cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [14]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "--n_estimators": choice(5,10,15,20,25),
    "--max_depth":choice(range(3,11)),
    "--min_samples_split": choice(2,5,8,12,15,18),
    "--random_state": choice(range(1,11))
})

script_folder = './'

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory = script_folder, 
              compute_target = cpu_cluster, 
              vm_size = 'STANDARD_D2_V2', 
              vm_priority = 'lowpriority',
              entry_script = 'train.py')

hyperdrive_run_config = HyperDriveConfig(estimator = estimator,
                                     hyperparameter_sampling = param_sampling,
                                     policy = early_termination_policy,
                                     primary_metric_name = 'Accuracy',
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                     max_concurrent_runs = 3,
                                     max_total_runs = 50)

In [12]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_8e9525d2-faf9-433a-af15-5037ec8c0bd5
Web View: https://ml.azure.com/experiments/Titanic_project/runs/HD_8e9525d2-faf9-433a-af15-5037ec8c0bd5?wsid=/subscriptions/2c48c51c-bd47-40d4-abbe-fb8eabd19c8c/resourcegroups/aml-quickstarts-136380/workspaces/quick-starts-ws-136380

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-28T13:49:11.685792][API][INFO]Experiment created<END>\n""<START>[2021-01-28T13:49:12.167265][GENERATOR][INFO]Trying to sample '3' jobs from the hyperparameter space<END>\n""<START>[2021-01-28T13:49:12.511369][GENERATOR][INFO]Successfully sampled '3' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-01-28T13:49:12.7326542Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_8e9525d2-faf9-433a-af15-5037ec8c0bd5
Web View: https://ml.azure.com/experiments/Titanic_project/runs/HD_8e9525d2-faf9-433a-af15-5037ec8c0bd5?wsid=/subscriptions/2c48c

{'runId': 'HD_8e9525d2-faf9-433a-af15-5037ec8c0bd5',
 'target': 'ML-cluster-uda',
 'status': 'Canceled',
 'startTimeUtc': '2021-01-28T13:49:11.368243Z',
 'endTimeUtc': '2021-01-28T13:57:19.42438Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name": "

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print(best_run.get_details()['runDefinition']['arguments'])

print('Id of the best run: ', best_run.id)
print('Accuracy of the best run:', best_run_metrics['Accuracy']) # selvitä muut argumentit
print('\n n_estimators:',parameter_values['n_estimators'])
print('\n max_depth:',parameter_values['max_depth'])
print('\n min_samples_split:',parameter_values['min_samples_split'])
print('\n random_state:',parameter_values['random_state'])

In [ ]:
#TODO: Save the best model
best_run.register_model(model_name = 'best_hyperdrive_model', model_path = 'outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service